A small step backward
---------------------------

<img src=https://github.com/cocteau/D4D/raw/master/images/Paper-stack.jpg width=400>

Today we are going to do a bit of web work, rehearsing a task that seems to still be discouragingly common -- web scraping. We'll wrap the lesson up in a general "this is how you might encounter data on the web" lesson. And so we will also talk about APIs briefly. Next time, we will see how we can deal with the text of a web page or a document "as data" in a bit more depth. 

Before we start the real work today, I wanted to take a step back and talk about some of the syntax of R that predates `dplyr` and those handy verbs like `mutate()` and `summarize()`. A data frame is a pretty basic object in R and we've seen a few of them now. Let's read in the MCCA data again briefly.

In [ ]:
chiefs <- read.csv("https://github.com/cocteau/D4D/raw/master/data/mcca.csv")
head(chiefs)

**Extracting data**. First, while we have been using `select()` from `dplyr`, we can also extract columns from the data frame using "$" the dollar sign...

In [ ]:
chiefs$agency

... and the vaues for homicides in 2016.

In [ ]:
chiefs$hom16

We can also access the elements of  a data frame directly using square brackets. There are several different expressions here, each handy at different times. For example, the robbery statistics from 2016 are in column number 6. We can pull them with either `chiefs$rob16` or...

In [ ]:
chiefs[,6]

The comma separates our row selection (to the left) from our column selection (to the right). When there are no choices made, R will assume you want everything. So `chiefs[,6]` is all the rows from the table with data just from the 6th column. By contrast, the expression below looks at all the data from the 20th agency...

In [ ]:
chiefs[20,]

... and chiefs[15,1:3] takes the first three columns from the 15th agency. (The ":" operator creating sequences of integers.)

In [ ]:
chiefs[15,1:3]

What does this do?

In [ ]:
chiefs[5:10,c(1,6:9)]

In the above, we have used c(...) to concatenate some integers together to help us select the first column along with numbers 6, 7 8 and 9.

In [ ]:
c(1,6:9,5,23,6)

We might reasonably ask about which cities had more than 300 robberies in 2016. We can use a "logical expression" for this. It will return a set of boolean data types, TRUE and FALSE. Ah we've seen these before! I present this because it is what is under the hood when we use `filter()` from `dplyr` -- we are keeping rows that have `TRUE` and leaving out those that are `FALSE`.

In [ ]:
chiefs$rob16 > 300

Notice we still have our NA or missing value. That city that caused us problems when we computed robberies is still missing. (That's a good thing because R keeps NA's around -- if robbery data is missing, the total number of violent crimes should be missing too.)  We can ask which crime statitics are NA by simply asking...

In [ ]:
is.na(chiefs$rob16)

**Data "out there"**


Documents are part of your everyday practice. Whether from a FOIA request or "scraped" from the web, you will inevitably have to spend time with a raft of text files. Sometimes, it's sufficient to simply read them and, well, take notes. Those notes, your observations about the people or places or events in the documents, can become the basis for further reporting. Depending on the number of documents and their content, you might want to be more systematic in your note taking. You might want to track specific aspects of your documents, perhaps looking for patterns among the people involved, or over time, or across cities. This suggests taking what is referred to as "unstructured" data and turning it into something "structured." 

Contrast [the web page of Obama's commutations](https://www.justice.gov/pardon/obama-commutations) with the various data frames we've examined in R from the UCR Program or the MCCA, say. This is a list of people granted pardons and commuations under [the clemency initiative](https://www.justice.gov/pardon/clemency-initiative). As a web page, the list of commutations are meant to be read. You can visually scan the page, or perhaps search the page for years (like 2008) or places (like Iowa). Our data frames, on the other hand, let us immediately tally up how many rows match a given pattern. We can perform computations on each column, examine subsets of the data, produce informative summary graphics, and fit models to cluster rows that have similar characteristics.

So how do we take the free text of [the commutations web page](https://www.justice.gov/pardon/obama-commutations) and systematically fill in a more structured data set? To be concrete, let's consider making a data frame where each row is a commuted sentence. What variables do we want to record for each commutation and how do we identify the information on the web page?

As an exmple, consider the "District/Date" entries. Here are a few.

>District of Connecticut; April 21, 2009
<br>District of South Carolina; February 21, 2008
<br>Eastern District of Virginia; June 19, 2008
<br>Western District of Michigan; December 18, 2006

For each commutation, we might want to split the "where" and the "when" and then populate two columns in a data frame. Now look at a few entries in the "Sentences" rows.

>360 months' imprisonment; eight years' supervised release
<br>292 months' imprisonment; five years' supervised release
<br>240 months' imprisonment; five years' supervised release
<br>Life imprisonment; six years’ supervised release

Here things are a little different. Still we have a pattern that emerges in each description (jail time then supervised release), but the language might vary -- from months to years to a category like "Life imprisonment." And with documents that evolve in time, there are inevitably shifts in formats and categories. The more time that passes from the first document in your collection to the last, the greater the chance that the structure of the documents changes. 

And that's the case here. In your browser, have a look at the "source" of the [commutations page](https://www.justice.gov/pardon/obama-commutations) (in Chrome, you select the "View" tab and then "Developer" and then "Source"). Search for "&lt;table". This will take you through the "tables" on the page. There's one table of pardons after an "h2" header with text "January 17, 2017" and another table after an "h2" header with text "December 19, 2016". 

Today we are going to divide the task of finding and organizing data on the web into two separate activities. Our focus for this session will be finding data and pulling it into R and next time we will look at how to put structure on what we find -- that will mean simple text serches to a more elaborate pattern language. 

We are going to look at a couple examples today, our first being the [commutations web page for Obama](https://www.justice.gov/pardon/obama-commutations), which means we'll start by talking about how we work with text coming from an HTML page.

**"Scraping" data from the web**

[The commutations page from the Justice Department](https://www.justice.gov/pardon/obama-commutations) stores the data we are after in the form of an HTML table. We need to pull the data out and load it into R. For this, we need another package, `rvest`.

Let's load it up and kick the tires. We also load `dplyr` because eventually we're going to create a data frame! And, as you recall, all the `dplyr` functions take data frames as arguments and manipulate them in some way.

In [ ]:
library(rvest)
library(selectr)
library(dplyr)

`rvest` contains a number of functions to bring web pages into R, making them expressive. By that I mean, you have the capacity to search for various tags and identify content. Hadley Wickham has written  [a simple description of rvest](https://github.com/hadley/rvest).

Now, to specify what you want to extract from a web page, the browser plugin SelectorGadget is extremely handy. It's use is documented here. Please look over the documentation and follow along. It will have you install SelectorGadget and show you how it works.

> [https://selectorgadget.com/](https://selectorgadget.com/)

Why do we need something like SelectorGadget? Or why do I think it's awesome? Well, HTML, the HyperText Markup Language, is designed to format information as a document. So there are "tags" to define paragraphs and headings and lists. When we publish data like the commutations as a web page, in HTML, details about each commutation might be formatted as an element in a table or a list item, and perhaps be associated with a particular font choice or specifying the text as "bold". We make these decisions based on how we think about the web page and how it will look.

On the flip side, when you are given information in HTML as a web page, then, the document tags and styling often become important tools to help us reverse engineer what was in the mind of the designer, helping us to identify  how the details of each commutation were formatted. 

So, SelectorGadget examines a web page and produces a "CSS selector" that identifies just the data you want. CSS selectors are [described here](https://www.w3schools.com/cssref/css_selectors.asp) and are tools for associating styling with tags. Do you want headings a certain color or rendered in a certain font? Selectors let you specify that. Our task is to reverse engineering this logic, using the tag names and their styling to pull the data hiding in a web page. To bone up on CSS selectors, you might also try [this amusing game](http://flukeout.github.io/).

This act of pulling or "scraping" data from web pages can be hard. There are plenty of online tools that do it now, with varying degrees of success. To get us started, we'll pull just the dates of **Obama's pardon announcements.** (The page is smaller and a little easier to work with than the much larger commutations.) They are listed at the top of the [Obama pardons page](https://www.justice.gov/pardon/obama-pardons) and we can use SelectorGadget to find their "CSS selector," the assignment of styling to tags, that gets us the dates we need.

In [ ]:
page <- read_html("https://www.justice.gov/pardon/obama-pardons")

The function `html_nodes()` extracts various components from the web page. Here, for example, are all the `h2` headings and then all the anchor `a` tags.

In [ ]:
html_nodes(page,"h2")

In [ ]:
html_nodes(page,"a")

Take a moment to appreciate what we've done. The `read_html()` function has returned an object that we've called `page`. It has brought the pardons web page into R in a way that we can program with. We have immediate hooks into all the elements on the page. We can extract any portion of the page we want by specifying a tag name and its styling, essentially, through a CSS selector.

For the pardon dates, the nodes we are after are also anchor tags. But, they are anchor tags that are contained in a paragraph tag &lt;p&gt; having a `class` attribute of `rtecenter`. Search the source of the pardons page for the string `rtecenter` and you'll see it appears in just one place on the page. 

>&lt;p class="rtecenter"&gt;&lt;a href="#PJAN172017"&gt;January 17, 2017&lt;/a&gt&gt; | &lt;a href="##PardonDec192016"&gt;December 19, 2016&lt;/a&gt; | &lt;a href="#jan2016"&gt;January 16, 2016&lt;/a&gt; | &lt;a href="#dec2015"&gt;December 18, 2015&lt;/a&gt; | &lt;a href="#dec2014"&gt;December 17, 2014&lt;/a&gt; | &lt;a href="#dec2013"&gt;December 19, &gt; | &gt;March 1, 2013&lt;/a&gt; | &lt;a href="#nov"&gt;November 21, 2011&lt;/a&gt; | &lt;a href="#may"&gt;May 20, &lt;/a&gt; | &lt;a href="#dec"&gt;December 3, 2010&lt;/a&gt;&lt;/p&gt;

OK that's a little ugly, but you see the &lt;p&gt; tag and then a series of &lt;a&gt; tags that contain our dates. So, we can use a CSS selector to specify this. We want someting of class `rtecenter` and then the anchor tags it contains. Thankfully,  SelectorGadget can get us the required selector directly by simply highlighting the data we want. SelectorGadget will return ".rtecenter a". Again, you can skim the resources above (2 minutes of work) to see what this means. 

For now, take the selector string and extract what we need from the "page".

In [ ]:
dts <- html_nodes(page,".rtecenter a")
dts

We can extract the text contained in each of the anchors &lt;a&gt; with a function `html_text()`...

In [ ]:
html_text(dts)

or get the href attributes using `html_attr()`.

In [ ]:
html_attr(dts,"href")

**1. From the web page of Clemency Recipients https://www.justice.gov/pardon/clemencyrecipients
use SelectorGadget to identify the CSS selector for the &lt;a&gt; anchor tags for the "Clemency Recipients" under the presidents going back to Nixon. Then use read_html() and html_nodes() to extract these links. There should be eighteen of them.**

In [ ]:
# Your code here


**Back to Obama's commutations.** Let's return to [Obama's commutations page](https://www.justice.gov/pardon/obama-commutations), as we observed above, the details of the commutations themselves are stored in tables. We can select all the tables using the CSS selector that specifies just "table" tags. Here we are looking for each &lt;table&gt; and don't care about the styling or class or any of that. So the selector is just the string "table".

In [ ]:
page <- read_html("https://www.justice.gov/pardon/obama-commutations")

tbs <- html_nodes(page,"table")
tbs

We access the various tables using another data type in R called a `list`. A list is just a bag of data, accessed sequentially - there is a first item, a second and so on. It can contain any kind of data you like. It is like a vector except that vectors can contain only one kind of data. Vectors can be all numbers or all strings or all `TRUE/FALSE`'s. A list can store data of any kind. Have a look. 

Here we make a list with four items. A vector of numbers, a string, a floating point number and a vector of `TRUE/FALSE`'s.

In [ ]:
x <- list(c(1,2,3), "hi", 3.56, c(TRUE,FALSE,TRUE))
print(x)

And the purpose of this diversion is to show you how to access data. A vector accesses data with square brackets [ ]. A list uses double square brackets [[ ]].

In [ ]:
x[[2]]

In [ ]:
x[[4]]

Just to drive this home, the fourth object in the list "x" is a vector of TRUE and FALSE values. As a vector, it can be indexed with single square brackets [ ]. So, if we want the second element of this vector, which is the fourth element of the list "x", we would do the following.

In [ ]:
x[[4]][2]

So with a list or a list-like object, the double brackets get us to individual data items. 

Back to our tables. We also get at individual tables in "tbs" using  square brackets. So `tbs[[1]]` gives us the first table in HTML format, and `tbs[[2]]` gives us the second. 

The function `html_table()` takes a single table and "parses" it into a data frame. This should look familiar! We do the first and tenth HTML tables on the commutation web page below.

In [ ]:
t1 <- html_table(tbs[[1]])
t1

In [ ]:
t10 <- html_table(tbs[[10]])
t10

Ha! Something familiar! We can use the `dim()` function to tell us how many rows are in each data frame.

In [ ]:
dim(t1)

In [ ]:
dim(t10)

Now, this does not mean 1660 or 210 commutations. This means 1660 different cells (marked off as yellow or white rows in the commutations web page) in the first table. See that the first commutation in the tenth table takes up the first five rows of `t10`.

Also notice that because the HTML page didn't include table headers, the data frames `t1` and `t10` have default column names `X1` and `X2`. We can give them more descriptive names using the `rename()` function from `dplyr.` 

In [ ]:
t1 <- rename(t1,item=X1,description=X2)
head(t1)

And finally, we can start to work with the data a little. Here are all the sentences from the first table. We can use "dplyr" to select just the rows there the "item" entry is "Sentence:". Notice how we are still using text that is formatted to be read, as a document. It's "Sentence:" (the word sentence and then a colon) and not just "Sentence".

In [ ]:
sents <- filter(t1,item=="Sentence:")
sents

**2. Create a new data frame called dd (for district/date) that contains just the rows of "t1" that refer to the district and date of conviction.**

In [ ]:
# Put your code here



**The bigger picture**

Now, we are going to iterate (loop) over all the tables in "tbs" that we extracted from the commutations page and create one large data frame. We will start with `t1` and then loop over the second, third, fourth and so on tables. We will let `i` hold the table number and then let `i` take on the value 2, 3, 4... all the way up to the `length(tbs)`. 

In the code below, we do just what we did to make `t1` with the exception of a subsetting step. We take `tt` below and consider just the first two columns. We do this because the 9th table on the page turns out to have 3 columns and not 2. Look for the name `Saboonchi` in the source of the commutations web page and you'll see 3 &lt;td&gt; tags in one row. Again, my friends, data cleaning.

For each table, we then rename the columns as we did for `t1` and `bind_rows()` from `dplyr` them to a growing data frame called `commutations`. The `bind_rows()` function takes the first data frame argument and adds the second to the end of it. 

In [ ]:
# start with t1 and then loop through the other tables, adding to
# the big "commutations" data set

commutations <- t1

for(i in 2:length(tbs)){
    
    tt <- html_table(tbs[[i]])
    
    # Rename the columns from X1 and X2
    
    tt <- rename(tt,item=X1,description=X2)
    
    # Add the current table to the end of "commutations" by row-binding them
    
    commutations = bind_rows(commutations,tt)
}

In [ ]:
head(commutations)

In [ ]:
tail(commutations)

In [ ]:
dim(commutations)

Let's look at the values in the "item" column. It has things like "District/Date:" and "Sentence:" and, it turns out, some other things. We also see that we'll have to be careful as "Terms of Grant:" and "Terms of gran:", say, are both entries. This is what we call data cleaning.

In [ ]:
count(commutations,item)

Here we keep just district and date. We store it in `dd` which we see has 1713 entries. We also see that some tables don't have District/Date -- how many are missing?

In [ ]:
dd <- filter(commutations,item=="District/Date:")
dim(dd)

In [ ]:
head(dd)

In [ ]:
library(lubridate)
library(stringr)
library(ggplot2)

Now, we are going to use `mutate()` to add a column to `dd.` It will be formed by extracting the date portion of the `description`'s. We pull the date part by looking for everything after a semi-colon. Looking at the dates above, this seems like a good guess. We then pass that portion of the description onto `mdy()` to turn it into a date object.

Oh and in this case the regular expression specifies a semicolon followed by a space and then any character. The period "." is a wild card. 

In [ ]:
dd <- mutate(dd,dates=mdy(str_extract(description,"; .+")))
head(dd)

Let's look at the dates that failed to parse. They are returned as NA's.

In [ ]:
filter(dd,is.na(dates))

Now we see that the date might involve multiple dates! We'll need to clean these up manually. For the moment, let's make a histogram of the dates. 

In [ ]:
ggplot(dd,aes(x=dates))+geom_histogram(color="white")

Read the conditions of the [Clemency Initiative](https://www.justice.gov/pardon/clemency-initiative) and tell me if anything seems out of sorts with these dates.

**An example from class.** Diana was interested in sports offered to high school students in New York City. There have been various complaints that students of color and poorer students have been ["neglected by the PSAL (Public Schools Athletic League)."](https://www.scribd.com/document/140465589/The-Inequitable-Distribution-of-Sports-Funding-by-the-PSAL-Public-Copy) The PSAL is charged with overseeing the distribution of funds for sports programs. One old-ish report studied which schools received more support for different programs...

>Data Sources and Methodology
<br><br>All datawerecollected from April 2012 to June 2012.The PSAL official website was used to determine which schools had PSAL funded teams. The New York City High School Book was used to determine the borough, student enrollment, and campus affiliation,if any, of all schools. The NYC Department of Education website’s “find a school function” was used to determine which schools were on a campus. The New York City Department of Education’s School Facility Summary Website was also used to determine which schools were part of a campus.

The PSAL web site is at

> [http://www.psal.org/](http://www.psal.org/)

**3. To repeat the author of this study's method, what do we have to do? Map out a plan.**


Put your plan here.


**Scraping PSAL.** Let's give this a shot. We see that the team lists are spread across several HTML pages, indexed by the "Sports" tab in the upper lefthand corner of the page. That means we will need a loop again, but this time over sports. Have a look at the source and see what the links look like. As an example, the teams and schedules for baseball are here

>[http://www.psal.org/sports/sport.aspx?spCode=006&flag=All](http://www.psal.org/sports/sport.aspx?spCode=065&flag=All)

and for table tennis 

>[http://www.psal.org/sports/sport.aspx?spCode=065&flag=All](http://www.psal.org/sports/sport.aspx?spCode=065&flag=All)

Using our SelectorGadget skills we could extract the links to all the sports. Turns out they are attributes of anchor tags with class `lnk` that are contained in `div` tags with ID `spNav`. Here we request the PSAl home page and pull out the tags we want.

In [ ]:
library(rvest)

In [ ]:
page <- read_html("http://www.psal.org/")
sportlinks <- html_nodes(page,"#spNav .lnk")
sportlinks

And as we did before, we can pull the actual URLs, as they are just `href` attributes of these tags. 

In [ ]:
sports <- data.frame(sport=html_text(sportlinks),url=html_attr(sportlinks,"href"))
sports

Now, let's pull the baseball teams. We'll go their page and pull the teams' data. Again, looking at the page source, we see that this comes from a table on the page in the lower right. It's the third table on the page.

In [ ]:
page <- read_html("http://www.psal.org/sports/sport.aspx?spCode=006&flag=All")
tables <- html_table(page)

Remember, this function returns a `list` of tables. By inspection we want the third one on the page. 

In [ ]:
class(tables)

And here are all the schools with baseball teams. Notice that the table on the PSAL page has proper &lt;th&gt; header tags and so our data frame is even named properly!

In [ ]:
tables[[3]]

And from here we would create a loop and build up all the different sports. We would probably add the name of the sport as we go, using each URL in turn from our data frame of sports names and URLs.

In [ ]:
teams <- tables[[3]]
teams <- mutate(teams,SPORT="Baseball")
head(teams)

**Web Services and APIs**

What we are doing in the last example is using a web page as a carrier of data we're after. We are using an HTTP request as a kind of database lookup. The data we want, in this case teams for a sport, is specified by parameters in the URL. This method, by the way, is called GET. When the parameters we are sending are hidden in the payload of the request (like our headers) we call that POST. 

Here's access to the [Google Translate service.](http://translate.google.com). You can type in an expression and look at the URL change. Pass it a string, as well as the "from" and "to" languages will get back a translation.

>[https://translate.google.com/#view=home&op=translate&sl=en&tl=de&text=Data%20are%20fun!](https://translate.google.com/#view=home&op=translate&sl=en&tl=de&text=Data%20are%20fun!)

This idea of pulling data from a web server, executing a service and receiving output, has been formalized and made a lot friendlier to programmers. You can think of data now as a kind of service. Data are served up in machine-readable forms rather than web pages that have to be wrangled — an **Application Programming Interface or API** describes how you interact with a data server, how you pose queries and enforces constraints on how much data you can pull (why?). Here is a simple API for Google's toolbar suggestions. Notice that with a little string manipulation, we can test any starter search.

> [http://suggestqueries.google.com/complete/search?client=toolbar&q=why%20is](http://suggestqueries.google.com/complete/search?client=toolbar&q=why%20is)

The data come back in XML. It is one of the popular forms for an API to use. XML or the eXtensible Markup Language is in the same family as HTML, except that instead of having fixed tags, the tags are open to your choosing -- they give you freedom to describe your data. Much in the same way HTML tags describe components of a document.

In this case, the data we're after is stored in the attributes of the tags, not as strings in the tags themselves. We extract attribute data from a Tag object using square brackets and the name of the attribute.

Through APIs, machines can exchange data, feeding the output of one service as the input of another. You will find APIs for specific data requests like the auto-suggest as well as for realtime services like Twitter. The API is an interface in the sense that you just have to know what parameters it takes and then what format the data will emerge in. Typically, the formats are machine digestible like XML or JSON so that you don't have to work as hard as we did pulling information out of an HTML page.

**Homework**

We now have a fair number of skills to work with. I've created a data set from the PSAL web site and other sources that describe each school's sports offering, how many students they serve (well, the enrollment in the school) and how much money is alloted by the city for each sport. *If I did this correctly* there should be plenty to investigate here. 

For example, which sport is offered most often?

In [ ]:
psal <- read.csv("https://github.com/cocteau/D4D/raw/master/data/PSAL4.csv")
head(psal)

In [ ]:
count(psal,SportName)

In [ ]:
arrange(count(psal,SportName),desc(n))

Have a look at the data set. Pose three questions and then answer them with a mix of computation or graphics. Remember, you can aggregate by school or borough, collecting total costs of sports programs or the total number of students per sport offered. Consult the report I pointed you to in the PSAL section of this notebook.

Also I would encourage you to go through [this ggplot2 tutorial](https://opr.princeton.edu/workshops/Downloads/2018Jan_ggplot2KOffman.pdf). You don't need to install `ggplot2` so ignore that step. (You will have to use `library(ggplot2)` however as we have in the past to load it into your workspace.)